In [55]:
#!pip install cvxopt

In [56]:
# Import necessary libraries
import numpy as np
import pandas as pd

import yfinance as yf

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py  
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd
import pandas_datareader as dr

np.random.seed(123)

# Não print o progresso da otimização
solvers.options["show_progress"] = False

import warnings
warnings.filterwarnings("ignore")

In [57]:
tickers = ["CSNA3.SA", "PETR4.SA", "BBAS3.SA", "ITUB4.SA", "BBDC4.SA"]

ativos = []
for ticker in tickers:
    try:
        cotacoes = dr.DataReader(ticker, "yahoo", "01/01/2021")["Adj Close"]
        cotacoes = pd.DataFrame(cotacoes)
        cotacoes.columns = [ticker]
        ativos.append(cotacoes)
    except:
        pass
    base_ativos = pd.concat(ativos, axis = 1)
base_ativos.sort_index(inplace = True)
base_ativos.head()

,CSNA3.SA,PETR4.SA,BBAS3.SA,ITUB4.SA,BBDC4.SA
Date,,,,,
2021-01-04,29.011297,13.732212,32.590824,28.665434,20.408911
2021-01-05,29.537693,14.268963,32.252686,28.479900,20.284470
2021-01-06,30.785763,14.297462,32.989651,29.314817,20.956762
2021-01-07,32.713058,14.724960,34.298836,30.455870,21.513140
2021-01-08,32.908333,14.781960,34.498245,30.446587,21.327681


In [58]:
# Ficamos apenas com os retornos
base_ativos = pd.DataFrame(base_ativos.pct_change())
base_ativos.dropna(inplace = True)

In [59]:
base_ativos

,CSNA3.SA,PETR4.SA,BBAS3.SA,ITUB4.SA,BBDC4.SA
Date,,,,,
2021-01-05,0.018145,0.039087,-0.010375,-0.006472,-0.006097
2021-01-06,0.042253,0.001997,0.022850,0.029316,0.033143
2021-01-07,0.062603,0.029900,0.039685,0.038924,0.026549
2021-01-08,0.005969,0.003871,0.005814,-0.000305,-0.008621
2021-01-11,-0.021930,-0.008355,-0.016336,-0.022547,-0.017754
...,...,...,...,...,...
2022-12-09,0.051847,-0.002825,-0.001427,-0.009519,-0.017182
2022-12-12,-0.030385,-0.032376,-0.034019,-0.009453,-0.011189
2022-12-13,-0.033426,-0.024676,-0.047376,-0.037759,-0.029703


In [60]:
# Por causa da multiplicação de matrizes, você precisa converter o dataframe em array e transpor
np.array(base_ativos).T

array([[ 0.01814451,  0.04225346,  0.06260347, ..., -0.03342615,
         0.0345821 , -0.0229805 ],
       [ 0.03908699,  0.00199725,  0.02990033, ..., -0.02467587,
        -0.07933106,  0.02654875],
       [-0.01037527,  0.02284973,  0.03968472, ..., -0.04737633,
        -0.02479601,  0.02832318],
       [-0.00647238,  0.02931601,  0.03892408, ..., -0.03775933,
         0.00431222,  0.00558176],
       [-0.00609739,  0.03314322,  0.02654882, ..., -0.02970297,
         0.02113702, -0.0121342 ]])

In [61]:
fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)


for i in range(0, base_ativos.shape[1]):
    fig.add_trace(go.Scatter(x = base_ativos.index, y = base_ativos.iloc[:,i]*100
                         , name = base_ativos.columns[i], line = dict(color = "Green", width = 1))
              , row = 1, col = 1)


fig.update_layout(height = 800, width = 1000
                  , title_text = "Retornos da cesta de ativos"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Returns (%)"
                  , legend_title = "Ativos"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                  , showlegend = True
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = base_ativos.index[0]
                       , end = base_ativos.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in base_ativos.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)

fig.show()

In [62]:
# Função que cria pesos aleatórios

def pesos(n):
    w = np.random.rand(n)
    return w / sum(w)

print (pesos(base_ativos.shape[1]))

[0.28080675 0.11536743 0.09146337 0.2222825  0.29007995]


In [63]:
def portfolio_return(returns):

    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(pesos(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # Reduz outliers
    if sigma > 2:
        return portfolio_return(returns)
    return mu, sigma

In [64]:
n_portfolios = 1000

means, stds = np.column_stack([portfolio_return(np.array(base_ativos).T) for _ in range(n_portfolios)])

In [65]:
# Plotando a média e o sigma dos portfolios criados

fig = go.Figure(data = go.Scatter(x = stds[:,0], y = means[:,0], mode = "markers", marker_color = "blue"))

fig.update_layout(height = 600, width = 800
                  , title_text = "Média e sigma dos portfolios criados"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Risco"
                  , yaxis_title = "Retornos"
                  , font = dict(size = 15, color = "Black")
                 )

fig.show()


In [66]:
def portfolio_otimo(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 400
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Converte para matrizes no formato cvxopt
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Cria as matrizes de restrições
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calcular pesos de fronteira eficientes usando programação quadrática
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## Calcula os riscos e retornos da fronteira
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    
    ## Calcula o polinômio de 2º grau da fronteira
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    
    # Calcula o portfolio ótimo
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks

In [67]:
weights, returns, risks = portfolio_otimo(np.array(base_ativos).T)

# Plotando a média e o sigma dos portfolios criados

#fig = go.Figure(data = go.Scatter(x = stds[:,0], y = means[:,0], mode = "markers", marker_color = "blue"))
#fig = go.Figure(data = go.Scatter(x = risks, y = returns, mode = "lines+markers", marker_color = "blue"))
fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = stds[:,0], y = means[:,0], mode = "markers", marker_color = "blue"
                        , name = "Risco x Retorno")
              , row = 1, col = 1)
fig.add_trace(go.Scatter(x = risks, y = returns, mode = "lines+markers", marker_color = "green"
                        , name = "Fronteira Eficiente")
              , row = 1, col = 1)


fig.update_layout(height = 600, width = 800
                  , title_text = "Fronteira eficiente"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Risco"
                  , yaxis_title = "Retornos"
                  , font = dict(size = 15, color = "Black")
                  , showlegend = True
                 )

fig.show()


In [68]:
np.round(weights*100, 4)

array([[  0.],
       [100.],
       [  0.],
       [  0.],
       [  0.]])